# Hotel Room Booking Data Preprocessing

### Notebook Introduction: 
This Python Notebook is a part of Hotel Room Booking Data Analysis Project, in which the original dataset will be cleaned and transformed into 2 tables for further analysis and visualizations. Also, a quick and simple EDA will be done to have a grasp about our data. 

The features of 2 new tables will be carefully selected and preprocessed to answer a series of busienss questions:

1. What are the total reservations and total revenue from room bookings?
2. How was the ADR changed through out the years?
3. Do City Hotel and Resort Hotel perform differently? If so, how?
3. What are the types of our major customers and through what channels do they book with us?

### Workflow
**1. Load original Data**  
**2. Data Cleaning & Manipulation:**
   - Handle duplicates
   - Handle missing values 
   - Restructure the data
   
**3. EDA:**
   - Explore the major columns 
   - Add new columns of KPIs and important measures
   - Export tables

#### Part 1 Load Original Data

In [1]:
#import libraries
import pandas as pd

In [2]:
#load the original data
df = pd.read_csv('/Users/kouyuyang/Documents/Projects/Kaggle Projects/Hotel Demand Prediction/hotel_bookings.csv')
#sanity check
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [3]:
df.shape

(119390, 32)

In [4]:
#check the dataframe information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

**Comment:** We can see that there are 119390 rows and 32 columns. Each record is a hotel reservation record, including plenty of important infomration about reservation status, dates, customer information, stay-in nights, reservation channels and so on. 

Now let's check if there are duplicated rows and missing values. 

#### Part 2: Data Cleaning

##### 2.1 Handle Duplicates

In [5]:
#check if there are duplicated rows
df.duplicated().sum()

31994

**Comment:** Unfortuanately, there are 31994 duplicated rows, but how should we deal with them? Are they really duplicated records or there might be other explainations. For example, is it possible a group of people booked more than 1 room so two reservation record showed identical same. We can take a futher look at how `adult` column looks like in those duplicated rows. 

In [6]:
df[df.duplicated()]['adults'].value_counts()

2    25183
1     6524
3      267
0       18
4        2
Name: adults, dtype: int64

**Comment:** Most guests consist of more than one adult, so there are possibilities that they book more than one room. Even for records showing 1 adult or 0 adults, there might be a need to book extra rooms for their children or babies. Therefore, at this point, I would not drop the duplicated rows because, first, we need to base our assumption on the integrity of the data, and second, there isn't enough evidence to ensure that these duplicated rows are indeed duplicated records. 
Now, let's check the missing values. 

##### 2.2 Handle Missing Values

In [7]:
df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

Obviously, the columns of `agent`, `company`,`children`,and `country` have missing values. Let's look at it each by each. 

`agent`

In [8]:
df['agent'].value_counts()

9.0      31961
240.0    13922
1.0       7191
14.0      3640
7.0       3539
         ...  
289.0        1
432.0        1
265.0        1
93.0         1
304.0        1
Name: agent, Length: 333, dtype: int64

**Comment:** Firstly, without a detailed explanation, we have no idea what the column 'agent' means exactly here. Secondly, for analyzing booking sales and customer segments, we don't quite need the 'agent' column, so we can ignore the missing values and drop the 'agent' column later.

`company`

In [9]:
df['company'].value_counts()

40.0     927
223.0    784
67.0     267
45.0     250
153.0    215
        ... 
104.0      1
531.0      1
160.0      1
413.0      1
386.0      1
Name: company, Length: 352, dtype: int64

**Comment**: Similarly to the column `agent`, we don't have enough information about the meaning of these numbers, so we will drop this column later as well.

`children`

In [10]:
print(df['children'].unique())

[ 0.  1.  2. 10.  3. nan]


In [11]:
print(f"The percentage of missing values in children column is {df['children'].isna().sum()/len(df)*100}%")

The percentage of missing values in children column is 0.003350364352123293%


The extremely small percentage is nothing to worry. Moreover, it seems reasonable to assume that the absence of a recorded value implies the absence of accompanying children. Therefore, replacing these missing values (NaN) with 0 aligns with the assumption that no recorded value indicates zero children. 

In [12]:
#replace the missing value with '0'
df['children'] = df['children'].fillna(0)

In [13]:
#Sanity Check
df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               0
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

`country`

In [14]:
df['country'].unique()

array(['PRT', 'GBR', 'USA', 'ESP', 'IRL', 'FRA', nan, 'ROU', 'NOR', 'OMN',
       'ARG', 'POL', 'DEU', 'BEL', 'CHE', 'CN', 'GRC', 'ITA', 'NLD',
       'DNK', 'RUS', 'SWE', 'AUS', 'EST', 'CZE', 'BRA', 'FIN', 'MOZ',
       'BWA', 'LUX', 'SVN', 'ALB', 'IND', 'CHN', 'MEX', 'MAR', 'UKR',
       'SMR', 'LVA', 'PRI', 'SRB', 'CHL', 'AUT', 'BLR', 'LTU', 'TUR',
       'ZAF', 'AGO', 'ISR', 'CYM', 'ZMB', 'CPV', 'ZWE', 'DZA', 'KOR',
       'CRI', 'HUN', 'ARE', 'TUN', 'JAM', 'HRV', 'HKG', 'IRN', 'GEO',
       'AND', 'GIB', 'URY', 'JEY', 'CAF', 'CYP', 'COL', 'GGY', 'KWT',
       'NGA', 'MDV', 'VEN', 'SVK', 'FJI', 'KAZ', 'PAK', 'IDN', 'LBN',
       'PHL', 'SEN', 'SYC', 'AZE', 'BHR', 'NZL', 'THA', 'DOM', 'MKD',
       'MYS', 'ARM', 'JPN', 'LKA', 'CUB', 'CMR', 'BIH', 'MUS', 'COM',
       'SUR', 'UGA', 'BGR', 'CIV', 'JOR', 'SYR', 'SGP', 'BDI', 'SAU',
       'VNM', 'PLW', 'QAT', 'EGY', 'PER', 'MLT', 'MWI', 'ECU', 'MDG',
       'ISL', 'UZB', 'NPL', 'BHS', 'MAC', 'TGO', 'TWN', 'DJI', 'STP',
       'KNA', 'E

In [15]:
print(f"The percentage of missing values in country column is {df['country'].isna().sum()/len(df)*100}%")

The percentage of missing values in children column is 0.40874445095904177%


**Comment:** The missing data in the 'country' column is minimal, accounting for just 0.4%, thus not raising significant concerns. Furthermore, we can replace the 'nan' values with 'Unknown' to signify that the nationality of the customers is unidentified.

In [17]:
#replace the missing value with 'Unkown'
df['country'] = df['country'].fillna('Unknown')

#sanity check
df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               0
babies                                 0
meal                                   0
country                                0
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

##### 2.3 Split original table into 2 tables 

Now, let's decide how to divide the orginal table into more dimensions. Based on the business questions, we mainly need information about the room bookings and customer segements. Therefore, after careful selection, the original table will be restructured into 2 table: room_sales, room_customers. In order to join 2 tables easily, a column of reservation_id will be added to serve as the primary key. 


In [19]:
## Assign unique reservation_id to each reservation 
df['reservation_id'] = range(1, len(df) + 1)

## Sanity Check
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,reservation_id
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,1
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,2
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,3
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,4
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,5


In [20]:
# Get room_sales table
room_sales = df[['reservation_id','hotel','is_canceled', 'lead_time','arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adr', 'reservation_status','reservation_status_date']]

room_sales.head()

,reservation_id,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adr,reservation_status,reservation_status_date
0,1,Resort Hotel,0,342,2015,July,27,1,0,0,0.0,Check-Out,2015-07-01
1,2,Resort Hotel,0,737,2015,July,27,1,0,0,0.0,Check-Out,2015-07-01
2,3,Resort Hotel,0,7,2015,July,27,1,0,1,75.0,Check-Out,2015-07-02
3,4,Resort Hotel,0,13,2015,July,27,1,0,1,75.0,Check-Out,2015-07-02
4,5,Resort Hotel,0,14,2015,July,27,1,0,2,98.0,Check-Out,2015-07-03


In [21]:
#get room_customers table
room_customers = df[['reservation_id','adults','children','babies','meal','country', 'market_segment', 'distribution_channel', 'is_repeated_guest','reserved_room_type',
       'assigned_room_type','customer_type']]

#sanity check
room_customers.head()

,reservation_id,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,reserved_room_type,assigned_room_type,customer_type
0,1,2,0.0,0,BB,PRT,Direct,Direct,0,C,C,Transient
1,2,2,0.0,0,BB,PRT,Direct,Direct,0,C,C,Transient
2,3,1,0.0,0,BB,GBR,Direct,Direct,0,A,C,Transient
3,4,1,0.0,0,BB,GBR,Corporate,Corporate,0,A,A,Transient
4,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,A,A,Transient


#### Part 3 EDA

##### 3.1 room_sales

`hotel`

In [22]:
room_sales['hotel'].value_counts()

City Hotel      79330
Resort Hotel    40060
Name: hotel, dtype: int64

**Comment:** There are mainly two types of hotels: City Hotels and Resort Hotels. Typically, a city hotel is located within a city, while a resort hotel tends to be situated in areas with tourist attractions. Further exploration can be conducted.

`is_canceled`

In [23]:
room_sales['is_canceled'].value_counts()

0    75166
1    44224
Name: is_canceled, dtype: int64

**Comment:** Interestingly, the number of cancellations appears to be significant. 

`lead_time`

In [24]:
room_sales['lead_time'].describe()

count    119390.000000
mean        104.011416
std         106.863097
min           0.000000
25%          18.000000
50%          69.000000
75%         160.000000
max         737.000000
Name: lead_time, dtype: float64

**Comment:** The minimum lead time is 0 days, while the maximum lead time is 737 days. This indicates that some people choose to book their hotel on the day they check in, while others book their hotel more than 2 years in advance. The average lead time is 104 days, which is approximately 3 months in advance.

`arrival_date_year`,`arrival_date_month`, `arrival_date_week_number`,`arrival_date_day_of_month`

In [25]:
room_sales[['arrival_date_year','arrival_date_month', 'arrival_date_week_number','arrival_date_day_of_month']]

,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month
0,2015,July,27,1
1,2015,July,27,1
2,2015,July,27,1
3,2015,July,27,1
4,2015,July,27,1
...,...,...,...,...
119385,2017,August,35,30
119386,2017,August,35,31
119387,2017,August,35,31
119388,2017,August,35,31


**Comment**:In this project, these columns will be combined to generate the timestamp of the arrival date, as we will only focus on analyzing the general trend across different periods.

In [26]:
#map the month
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

room_sales['arrival_date_month'] = room_sales['arrival_date_month'].map(month_mapping)


/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/3191038659.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_sales['arrival_date_month'] = room_sales['arrival_date_month'].map(month_mapping)


In [27]:
#create a new column arrival_date
room_sales['arrival_date'] = pd.to_datetime({'year':room_sales['arrival_date_year'],
                                             'month':room_sales['arrival_date_month'],
                                             'day': room_sales['arrival_date_day_of_month']})
room_sales.head()

/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/3294596209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_sales['arrival_date'] = pd.to_datetime({'year':room_sales['arrival_date_year'],


,reservation_id,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adr,reservation_status,reservation_status_date,arrival_date
0,1,Resort Hotel,0,342,2015,7,27,1,0,0,0.0,Check-Out,2015-07-01,2015-07-01
1,2,Resort Hotel,0,737,2015,7,27,1,0,0,0.0,Check-Out,2015-07-01,2015-07-01
2,3,Resort Hotel,0,7,2015,7,27,1,0,1,75.0,Check-Out,2015-07-02,2015-07-01
3,4,Resort Hotel,0,13,2015,7,27,1,0,1,75.0,Check-Out,2015-07-02,2015-07-01
4,5,Resort Hotel,0,14,2015,7,27,1,0,2,98.0,Check-Out,2015-07-03,2015-07-01


In [28]:
#drop the columns
columns_to_drop = ['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']

room_sales.drop(columns=columns_to_drop, inplace=True)

#Sanity Check
room_sales.head()

/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/3134245405.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_sales.drop(columns=columns_to_drop, inplace=True)


,reservation_id,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adr,reservation_status,reservation_status_date,arrival_date
0,1,Resort Hotel,0,342,27,0,0,0.0,Check-Out,2015-07-01,2015-07-01
1,2,Resort Hotel,0,737,27,0,0,0.0,Check-Out,2015-07-01,2015-07-01
2,3,Resort Hotel,0,7,27,0,1,75.0,Check-Out,2015-07-02,2015-07-01
3,4,Resort Hotel,0,13,27,0,1,75.0,Check-Out,2015-07-02,2015-07-01
4,5,Resort Hotel,0,14,27,0,2,98.0,Check-Out,2015-07-03,2015-07-01


`arrival_date`

In [30]:
room_sales['arrival_date'].describe()

/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/2307047185.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  room_sales['arrival_date'].describe()


count                  119390
unique                    793
top       2015-12-05 00:00:00
freq                      448
first     2015-07-01 00:00:00
last      2017-08-31 00:00:00
Name: arrival_date, dtype: object

**Comment**: The arrival dates of reservations covered period from 2015-07-01 and 2017-08-31.

In [31]:
room_sales.columns

Index(['reservation_id', 'hotel', 'is_canceled', 'lead_time',
       'arrival_date_week_number', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adr', 'reservation_status',
       'reservation_status_date', 'arrival_date'],
      dtype='object')

`stay_in_weekend_nights`,`stats_in_week_nights`

We can get a total_stay_in_nights by summing these two columns

In [32]:
room_sales['total_stay_in_nights'] = room_sales['stays_in_weekend_nights'] + room_sales['stays_in_week_nights']

room_sales.head()

/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/450202365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_sales['total_stay_in_nights'] = room_sales['stays_in_weekend_nights'] + room_sales['stays_in_week_nights']


,reservation_id,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adr,reservation_status,reservation_status_date,arrival_date,total_stay_in_nights
0,1,Resort Hotel,0,342,27,0,0,0.0,Check-Out,2015-07-01,2015-07-01,0
1,2,Resort Hotel,0,737,27,0,0,0.0,Check-Out,2015-07-01,2015-07-01,0
2,3,Resort Hotel,0,7,27,0,1,75.0,Check-Out,2015-07-02,2015-07-01,1
3,4,Resort Hotel,0,13,27,0,1,75.0,Check-Out,2015-07-02,2015-07-01,1
4,5,Resort Hotel,0,14,27,0,2,98.0,Check-Out,2015-07-03,2015-07-01,2


In [34]:
room_sales['total_stay_in_nights'].describe()

count    119390.000000
mean          3.427900
std           2.557439
min           0.000000
25%           2.000000
50%           3.000000
75%           4.000000
max          69.000000
Name: total_stay_in_nights, dtype: float64

**Comment**:The shortest duration of stay is recorded as 0 days, whereas the longest stay spans 69 days. On average, guests typically stay for approximately 4 days.

`adr`

In [40]:
room_sales['adr'].describe()

count    119390.000000
mean        101.831122
std          50.535790
min          -6.380000
25%          69.290000
50%          94.575000
75%         126.000000
max        5400.000000
Name: adr, dtype: float64

**Comment**:The average of adr (averate daily rate) is about 102. 

`reservation_status`

In [35]:
room_sales['reservation_status'].value_counts()

Check-Out    75166
Canceled     43017
No-Show       1207
Name: reservation_status, dtype: int64

**Comment**: We can see almost 1/3 of guests cancelled their reservations and there are 1207 of no-show, which also indicates that the min of total_stay_in_nights is 0. 

`reservation_status_date`

In [39]:
#change the column to timestamp 
room_sales['reservation_status_date'] = pd.to_datetime(room_sales['reservation_status_date'])

room_sales['reservation_status_date'].describe()

/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/2051893762.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_sales['reservation_status_date'] = pd.to_datetime(room_sales['reservation_status_date'])
/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/2051893762.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  room_sales['reservation_status_date'].describe()


count                  119390
unique                    926
top       2015-10-21 00:00:00
freq                     1461
first     2014-10-17 00:00:00
last      2017-09-14 00:00:00
Name: reservation_status_date, dtype: object

**Comment**: The reservation_status_date is the check-out date on reservation, however many reservations were canceled so the starting date of 2014-10-17 is much earlier than the starting date of the arrival_date. We can take a glance to confirm. 

In [46]:
room_sales[room_sales['reservation_status_date']<'2015-01-01'].head()

,reservation_id,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adr,reservation_status,reservation_status_date,arrival_date,total_stay_in_nights
1545,1546,Resort Hotel,1,297,36,1,3,0.0,Canceled,2014-11-18,2015-09-03,4
73714,73715,City Hotel,1,265,28,0,2,62.8,Canceled,2014-10-17,2015-07-09,2
73715,73716,City Hotel,1,258,27,0,2,62.8,Canceled,2014-10-17,2015-07-02,2
73716,73717,City Hotel,1,258,27,0,2,62.8,Canceled,2014-10-17,2015-07-02,2
73717,73718,City Hotel,1,258,27,0,2,62.8,Canceled,2014-10-17,2015-07-02,2


**Comment**: We can see the reservation_status was 'Cancelled', which double confirms our previous guessing.

`room_revenue`

Lastly, we need a new column of room_revenue, which is calculated by multiplying the adr with the total_stay_in_nights. This column helps us know the profitability of the whole hotel group. 

In [49]:
room_sales['room_revenue'] = room_sales['adr']*room_sales['total_stay_in_nights']
room_sales.head()

/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/4053570076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_sales['room_revenue'] = room_sales['adr']*room_sales['total_stay_in_nights']


,reservation_id,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adr,reservation_status,reservation_status_date,arrival_date,total_stay_in_nights,room_revenue
0,1,Resort Hotel,0,342,27,0,0,0.0,Check-Out,2015-07-01,2015-07-01,0,0.0
1,2,Resort Hotel,0,737,27,0,0,0.0,Check-Out,2015-07-01,2015-07-01,0,0.0
2,3,Resort Hotel,0,7,27,0,1,75.0,Check-Out,2015-07-02,2015-07-01,1,75.0
3,4,Resort Hotel,0,13,27,0,1,75.0,Check-Out,2015-07-02,2015-07-01,1,75.0
4,5,Resort Hotel,0,14,27,0,2,98.0,Check-Out,2015-07-03,2015-07-01,2,196.0


In [50]:
#Final Sanity Check
room_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   reservation_id            119390 non-null  int64         
 1   hotel                     119390 non-null  object        
 2   is_canceled               119390 non-null  int64         
 3   lead_time                 119390 non-null  int64         
 4   arrival_date_week_number  119390 non-null  int64         
 5   stays_in_weekend_nights   119390 non-null  int64         
 6   stays_in_week_nights      119390 non-null  int64         
 7   adr                       119390 non-null  float64       
 8   reservation_status        119390 non-null  object        
 9   reservation_status_date   119390 non-null  datetime64[ns]
 10  arrival_date              119390 non-null  datetime64[ns]
 11  total_stay_in_nights      119390 non-null  int64         
 12  ro

**Comment**: The room_sales table has been well processed and ready to export for our further visualization analysis. 

In [66]:
# Export the DataFrame to a CSV file
room_sales.to_csv('/Users/kouyuyang/Documents/Projects/Kaggle Projects/Hotel Demand Prediction/split tables for BI/room_sales.csv', index=False)

#### 3.2 room_customers

In [51]:
room_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   reservation_id        119390 non-null  int64  
 1   adults                119390 non-null  int64  
 2   children              119390 non-null  float64
 3   babies                119390 non-null  int64  
 4   meal                  119390 non-null  object 
 5   country               119390 non-null  object 
 6   market_segment        119390 non-null  object 
 7   distribution_channel  119390 non-null  object 
 8   is_repeated_guest     119390 non-null  int64  
 9   reserved_room_type    119390 non-null  object 
 10  assigned_room_type    119390 non-null  object 
 11  customer_type         119390 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 10.9+ MB


`adults`,`children`,`babies`

In [56]:
room_customers[['adults','children','babies']].describe()

,adults,children,babies
count,119390.000000,119390.000000,119390.000000
mean,1.856403,0.103886,0.007949
std,0.579261,0.398555,0.097436
min,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000
75%,2.000000,0.000000,0.000000
max,55.000000,10.000000,10.000000


**Comment**: The maximum adults is 55 people, which could be a group party. 

`meal`

In [57]:
room_customers['meal'].value_counts()

BB           92310
HB           14463
SC           10650
Undefined     1169
FB             798
Name: meal, dtype: int64

**Comment**: The most popular type of meal is BB, which is short for bed and breakfast.

`market_segment`

In [59]:
room_customers['market_segment'].value_counts()

Online TA        56477
Offline TA/TO    24219
Groups           19811
Direct           12606
Corporate         5295
Complementary      743
Aviation           237
Undefined            2
Name: market_segment, dtype: int64

**Comment**: We can see the hotel has diversified tyeps of market segments, among which the majority of customers come from online TA. 

`distribution_channel`

In [60]:
room_customers['distribution_channel'].value_counts()

TA/TO        97870
Direct       14645
Corporate     6677
GDS            193
Undefined        5
Name: distribution_channel, dtype: int64

**Comment**: Similar to the market_segments, travel agency and tour operator is the main channel. 

`customer_type`

In [61]:
room_customers['customer_type'].value_counts()

Transient          89613
Transient-Party    25124
Contract            4076
Group                577
Name: customer_type, dtype: int64

**Comment**: Most room_customers are transients or transiet-party.

`is_repeated_guest`

In [62]:
room_customers['is_repeated_guest'].value_counts()

0    115580
1      3810
Name: is_repeated_guest, dtype: int64

**Comment**:Only 3810 are repeated guests. This may encourage the hotel to promote their loyal program to bring more repeated customers back. 

`total_guests`

We need a new column of total_guests, which sums all guests of a reservation, including the children and babies. 

In [63]:
room_customers['total_guests'] = room_customers['adults']+room_customers['children']+room_customers['babies']
room_customers['total_guests'] = room_customers['total_guests'].astype(int)
room_customers.head()

/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/447859946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_customers['total_guests'] = room_customers['adults']+room_customers['children']+room_customers['babies']
/var/folders/v2/fv31n9f13dg7wh_mnsttm03h0000gn/T/ipykernel_3566/447859946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room_customers['total_guests'] = room_customers['total_guests'].astype(int)


,reservation_id,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,reserved_room_type,assigned_room_type,customer_type,total_guests
0,1,2,0.0,0,BB,PRT,Direct,Direct,0,C,C,Transient,2
1,2,2,0.0,0,BB,PRT,Direct,Direct,0,C,C,Transient,2
2,3,1,0.0,0,BB,GBR,Direct,Direct,0,A,C,Transient,1
3,4,1,0.0,0,BB,GBR,Corporate,Corporate,0,A,A,Transient,1
4,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,A,A,Transient,2


In [64]:
#Sanity Check
room_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   reservation_id        119390 non-null  int64  
 1   adults                119390 non-null  int64  
 2   children              119390 non-null  float64
 3   babies                119390 non-null  int64  
 4   meal                  119390 non-null  object 
 5   country               119390 non-null  object 
 6   market_segment        119390 non-null  object 
 7   distribution_channel  119390 non-null  object 
 8   is_repeated_guest     119390 non-null  int64  
 9   reserved_room_type    119390 non-null  object 
 10  assigned_room_type    119390 non-null  object 
 11  customer_type         119390 non-null  object 
 12  total_guests          119390 non-null  int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 11.8+ MB


Now, room_customers has been cleaned and ready for further analysis. 

In [65]:
# Export the DataFrame to a CSV file
room_customers.to_csv('/Users/kouyuyang/Documents/Projects/Kaggle Projects/Hotel Demand Prediction/split tables for BI/room_customers.csv', index=False)